# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Load CSV file
csv_file = "/Users/ashleypatricia/Documents/GitHub/python-api-challenge/WeatherPy/city data.csv"

# Assign dataframe 
weather_df = pd.read_csv(csv_file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vagur,61.4732,-6.8119,43.21,71,100,3.87,FO,1619138020
1,chokurdakh,70.6333,147.9167,4.89,91,90,5.70,RU,1619138020
2,bluff,-46.6000,168.3333,57.00,74,100,8.01,NZ,1619137761
3,kalabo,-14.9700,22.6814,62.35,62,5,5.19,ZM,1619138021
4,bethel,41.3712,-73.4140,39.99,54,55,5.01,US,1619137915


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [25]:
# Use the Lat and Lng as locations 
locations = weather_df[["Lat", "Lng"]]

# Use humidity as weight 
humidity = weather_df["Humidity"].astype(float)

# Plot heatmap
fig = gmaps.figure()

# Add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer to plot 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
